In [ ]:
import gen as g
from tqdm import tqdm

In [ ]:
def generate():
    dead_farmers = 0
        
    recommended_species = None
    
    chosen_class = None
    
    background = None

    while not (recommended_species and chosen_class and background):
        stats, attempts = g.roll_stats()
        dead_farmers += attempts
        recommended_species = g.recommend_species(stats)  
        updated_stats = g.apply_species_bonus(stats, recommended_species)
        optimal_stats = g.sort_stats(updated_stats)    
        chosen_class = g.select_class(optimal_stats)    
        background = g.pick_background(optimal_stats)
    alignment = g.determine_alignment(dead_farmers)

    return recommended_species, chosen_class, background, alignment
    

In [ ]:
def track_species_selection(num_iterations):
    species_count = {}
    classes_count = {}
    background_count = {}
    alignment_count = {}
    for _ in tqdm(range(num_iterations)):
        species, chosen_class, chosen_background, alignment = generate()
        # print(species, chosen_class, chosen_background, alignment)
        # break
        if species in species_count:
            species_count[species] += 1
        else:
            species_count[species] = 1
            
        if chosen_class in classes_count:
            classes_count[chosen_class] += 1
        else:
            classes_count[chosen_class] = 1
            
        if chosen_background in background_count:
            background_count[chosen_background] += 1
        else:
            background_count[chosen_background] = 1
            
        if alignment in alignment_count:
            alignment_count[alignment] += 1
        else:
            alignment_count[alignment] = 1

    return species_count, classes_count, background_count, alignment_count

species_results, class_results, background_results, alignment_results = track_species_selection(10000)

for species, count in species_results.items():
    print(f'{species}: {count}')
    
for chosen_class, count in class_results.items():
    print(f'{chosen_class}: {count}')
    
for chosen_background, count in background_results.items():
    print(f'{chosen_background}: {count}')
     
for alignment, count in alignment_results.items():
    print(f'{alignment}: {count}')

In [ ]:
import matplotlib.pyplot as plt
sorted_species = sorted(species_results.items(), key=lambda item: item[1], reverse=True)
    
species, counts = zip(*sorted_species)

plt.figure(figsize=(10, 6))
plt.bar(species, counts)
plt.title('Species Selections For 10,000 Rolled Arrays', fontsize = 18)
plt.xlabel('Species', fontsize = 14)
plt.ylabel('Count', fontsize = 14)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--')
plt.tight_layout()
plt.savefig('results.png')

In [ ]:
##### import matplotlib.pyplot as plt
classes = sorted(class_results.items(), key=lambda item: item[1], reverse=True)
    
    # Unzip the sorted results into two lists
classe, counts = zip(*classes)

plt.figure(figsize=(10, 6))
plt.bar(classe, counts)
plt.title('Class Selections For 10,000 Rolled Arrays', fontsize = 18)
plt.xlabel('Species', fontsize = 14)
plt.ylabel('Count', fontsize = 14)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--')
plt.tight_layout()
plt.savefig('results.png')

In [ ]:
import matplotlib.pyplot as plt
backgrounds = sorted(background_results.items(), key=lambda item: item[1], reverse=True)
    
    # Unzip the sorted results into two lists
background, counts = zip(*backgrounds)

plt.figure(figsize=(10, 6))
plt.bar(background, counts)
plt.title('Background Selections For 10,000 Rolled Arrays', fontsize = 18)
plt.xlabel('Species', fontsize = 14)
plt.ylabel('Count', fontsize = 14)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--')
plt.tight_layout()
plt.savefig('results.png')

In [ ]:
import matplotlib.pyplot as plt
alignments = sorted(alignment_results.items(), key=lambda item: item[1], reverse=True)
    
    # Unzip the sorted results into two lists
alignment, counts = zip(*alignments)

plt.figure(figsize=(10, 6))
plt.bar(alignment, counts)
plt.title('Alignment Selections For 10,000 Rolled Arrays', fontsize = 18)
plt.xlabel('Alignment', fontsize = 14)
plt.ylabel('Count', fontsize = 14)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--')
plt.tight_layout()
plt.savefig('results.png')